In [1]:
library("btLoda")
library("histogram")
library("MASS")
library("dplyr")

Loading required package: histogram

Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
vpro = 20  # percentage of anomaly
alpha = vpro/100  # anomaly proportion in the mixture data set

print(alpha)

runs=2 ## 100 for actual experiments
comp_gt_perf=data.frame()
comp_perf1_e02=data.frame();comp_perf1_e05=data.frame();comp_perf1_e10=data.frame();
comp_perf1_loda_e02=data.frame();comp_perf1_loda_e05=data.frame();comp_perf1_loda_e10=data.frame();

[1] 0.2


###### datab: anomlay scores for the second clean data set

###### datan: anomaly scores for the first mixture data set

###### alpha: anomaly proportion

###### epsilon: which quantile of the anomaly score of anomlay points are we focusing on

###### nboots: number of bootstraps

### Functions to get quantile estimates in our setting

In [3]:
## RAW ECDF i.e. emperical cdf 
raw_cdf<- function(datab, datan, alpha, epsilon){
  trialv <- sort(c(datan, datab))
  F.n <- ecdf(datan)
  Fn  <- F.n(trialv)
  F.b <- ecdf(datab)
  Fb <- F.b(trialv)
  est=c()
  for (j in 0:5){ #   ## To get statistcs for over estimates[alpha_e] of \alpha    
    alpha_e=(1+0.02*j)*alpha   
  Fa <- (Fn - (1-alpha_e)*Fb)/alpha_e 
  if (length(which(Fa <= epsilon))==0){
    index <- 1
  }else{
    index <-max(which(Fa <= epsilon))
  }
  est[j+1]=trialv[index]
  }
  return (est)
}

## Isotonic Regression emperical CDF so that the CDF is Monotonic.

iso_cdf <- function(datab, datan, alpha, epsilon){
  trialv <- sort(c(datan, datab))
  F.n <- ecdf(datan)
  Fn  <- F.n(trialv)
  F.b <- ecdf(datab)
  Fb <- F.b(trialv)

  est=c()
  for (j in 0:5){
    alpha_e=(1+0.02*j)*alpha 
  Fa <- (Fn - (1-alpha_e)*Fb)/alpha_e 
  F.is = isoreg(Fa)$yf ## Computes the Isotonic Estimator of Fa
  F.is[which(F.is<=0)]=0  
  F.is[which(F.is>=1)]=1
  if (length(which(F.is <= epsilon))==0){
    index.is <- 1
  }else{
    index.is <-max(which(F.is <= epsilon))
  }
  est[j+1]=trialv[index.is]
  }
  return (est)
}

### Below function returns the Recall, False Positive Rate for both Raw cdf and Isotonized CDF for various values of alien proportions

In [4]:
## to use this function, for the mixture data set, the anomaly points need to be put in the beginning of the data set. 

cv_result <- function(datab, datan, alpha, epsilon){
  ## 10 fold cross validation
  nn <- length(datan)
  na <- round(nn*alpha) ## number of anomaly points. 
  if (nn/10 - floor(nn/10) > 0){
    group <- c(rep((1:10), floor(nn/10)), (1:(nn - 10*floor(nn/10))))
  }else{
    group <- rep((1:10), floor(nn/10))
  }
  group.label <- sample(group)

  ## build a data frame to help with grouping and identifying anomaly points
  ## the first column is for group id
  ## the second column is the row number of each point. All points with row number smaller or equal than na are actual anomaly points
  df <- data.frame(group.label = group.label, index = (1:nn))

  ## for direct ECDF
  correct.vec2 <- rep(0, 10) 
  wrong.vec2 <- rep(0, 10) 
  ano.vec2 <- rep(0, 10)
  nom.vec2 <- rep(0, 10)
  ## for isotonized ECDF
  correct.vec3 <- rep(0, 10)
  wrong.vec3 <- rep(0, 10) 
  ano.vec3 <- rep(0, 10)
  nom.vec3 <- rep(0, 10)

  # result <- list()
  result=data.frame()
  
  est.vec2 <- rep(0, 10)
  est.vec3 <- rep(0, 10)
 for (j in 0:5){
  alpha_e=0.002*j+alpha 
    for (i in 1:10){# cross validation
      use_datan <- datan[which(df$group.label!= i)] ## anomaly scores from the 9 folds to use
      test_datan <- datan[which(df$group.label == i)] ## anomaly scores from the left out 1 fold to use as testing data
      use_df <- df[which(df$group.label!= i),] ## the part of group label dataframe that correspond to the 9 folds
      test_df <- df[which(df$group.label == i),] ## the part of group label dataframe that correspond to the 1 fold
      index_use <- use_df$index ## index of the 9 folds
      index_test <- test_df$index ## index of the 1 fold
      est2 <- raw_cdf(datab, use_datan, alpha, epsilon) ## estimate from raw ECDF method
      est3 <- iso_cdf(datab, use_datan, alpha, epsilon) ## estiamte from isotonized ECDF method
         
      if (length(which(test_datan >= est2[j+1]))==0){
        correct.vec2[i] <- 0 ## number of correctly claimed anomaly points
        wrong.vec2[i] <- 0
      }else{
        pos2 <- index_test[which(test_datan >= est2[j+1])] ## the row numbers of the points in the 1 fold test data which are claimed to be anomaly
        correct.vec2[i] <- sum(pos2 <= na) 
        wrong.vec2[i] <- sum(pos2 > na) 
      }
      ano.vec2[i] <- sum(index_test <= na) 
      nom.vec2[i] <- sum(index_test > na) 
    
      if (length(which(test_datan >= est3[j+1]))==0){
        correct.vec3[i] <- 0 ## number of correctly claimed anomaly points
        wrong.vec3[i] <- 0
      }else{
        pos3 <- index_test[which(test_datan >= est3[j+1])] ## the row numbers of the points in the 1 fold test data which are claimed to be anomaly
        correct.vec3[i] <- sum(pos3 <= na) 
        wrong.vec3[i] <- sum(pos3 > na)
      }
      ano.vec3[i] <- sum(index_test <=na)
      nom.vec3[i] <- sum(index_test > na)  
      
      
    
     result1=rep(0,7)
     
     result1[1] <- sum(correct.vec2)/sum(ano.vec2) 
     result1[2] <- sum(wrong.vec2)/(sum(correct.vec2) + sum(wrong.vec2)) 
     result1[3] <- sum(wrong.vec2)/sum(nom.vec2) 
    
     result1[4] <- sum(correct.vec3)/sum(ano.vec3) 
     result1[5] <- sum(wrong.vec3)/(sum(correct.vec3) + sum(wrong.vec3)) 
     result1[6] <- sum(wrong.vec3)/sum(nom.vec3)
     
     result1[7]=alpha_e
     
     result=rbind(result,result1)
   }
  }
   return(result)
 }


### Below function returns average performance for all (100) runs 

In [5]:

overall_perf <- function(comb_perf,epsilon){
  raw_cdf_perf=comb_perf
  temp_ovr_perf=data.frame()
  
  for (i in 1:as.integer(ncol(raw_cdf_perf)/7)){

    temp_ovr_perf1=data.frame()  
    
    temp_ovr_perf1=c(
            mean(raw_cdf_perf[[1+(7*(i-1))]],na.rm=TRUE),sd(raw_cdf_perf[[1+(7*(i-1))]],na.rm=TRUE),#recall raw cdf
                    mean(raw_cdf_perf[[3+(7*(i-1))]],na.rm=TRUE),sd(raw_cdf_perf[[3+(7*(i-1))]],na.rm=TRUE),#FPR raw CDf
                    mean(raw_cdf_perf[[4+(7*(i-1))]],na.rm=TRUE),sd(raw_cdf_perf[[4+(7*(i-1))]],na.rm=TRUE),#recall iso cdf
                    mean(raw_cdf_perf[[6+(7*(i-1))]],na.rm=TRUE),sd(raw_cdf_perf[[6+(7*(i-1))]],na.rm=TRUE),#fpr iso cdf
                    mean(raw_cdf_perf[[7+(7*(i-1))]],na.rm=TRUE))#alpha used in estimation

    temp_ovr_perf=rbind(temp_ovr_perf,temp_ovr_perf1)
    
  }
  return(temp_ovr_perf)
}

# ## For confidence intervals based on 100 runs
error_tci=function(mean1,sd1,n2){
    error1=qt(0.975,df=n2-1)*sd1/sqrt(n2)
    return(error1)
}

## Overall Performance with Confidence intervals
overall_performance_ci <- function(overall_performance2,n1){
# n1=nrow(comb_perf)
    overall_performance3=data.frame()
    for (i in 1:nrow(overall_performance2)){
        error1a=error_tci(overall_performance2[i,1],overall_performance2[i,2],n1)#recall CI
        error1b=error_tci(overall_performance2[i,3],overall_performance2[i,4],n1)#1-ppv/FPR CI _rawcdf

        error1c=error_tci(overall_performance2[i,5],overall_performance2[i,6],n1)#recall CI
        error1d=error_tci(overall_performance2[i,7],overall_performance2[i,8],n1)#1-ppv/FPR CI _isocdf
        

        temp_perf3=c(
                         overall_performance2[i,1],error1a,overall_performance2[i,3],error1b,#recall,fpr rawcdf
                         overall_performance2[i,5],error1c,overall_performance2[i,7],error1d,#recall,fpr iso_cdf
                         overall_performance2[i,10],overall_performance2[i,11],overall_performance2[i,12],overall_performance2[i,9])
        overall_performance3=rbind(overall_performance3,temp_perf3)
    }
    return(overall_performance3)
}

###### Out of Bag estimation using Isolation Forest to estimate CDF of nominal data & train Iforest 
###### Here we are using 20% of the points for growing each tree, and grow 1000 trees
###### use randomly selected 20% of the points to grow each tree, 
###### for each point, use the average depth from the trees which don't use this point to calculate anomaly score
###### the following part is for getting anomaly scores for all clean data set, mixture data set and test data set. 

###### If experiments needs to be done on LODA too uncomment all LODA related code.

In [6]:
for (run in 1:runs){
  print("run")
  print(run) 
  ###########DATASETS
#   source('~/data_sets_load.R')
  source('~/R/workspace_r/jnrl papr exps/zz-trial_sep25/osu_iforest_trial/ICML-code/data_sets_trial.R')
  ##############################make these as list of dataframes###########################
table_freq=as.data.frame(table(data[[o_col]]))
freq_known_t=list()
data_known_t=list()
data_unknown_t=list()
no_known_class_inst=0
for  (j in 1:length(known_classes)){
   freq_known_t[[j]]=filter(table_freq,Var1==known_classes[j])[[2]][1];
   no_known_class_inst=no_known_class_inst+freq_known_t[[j]];
   data_known_t[[j]]=subset(data,data[[o_col]] %in% known_classes[j])
   data_unknown_t[[j]]=subset(data,data[[o_col]] %in% setdiff(tr,known_classes[j]))
   }

#############each class proportion
alpha_t=alpha
prob_tr=list()
alpha_class_t=list()
for  (j in 1:length(known_classes)){
  prob_tr[[j]]=freq_known_t[[j]]/no_known_class_inst;
  alpha_class_t[[j]]=1-((1-alpha_t)*prob_tr[[j]]);
}
##
  unknown_class=setdiff(tr,known_classes)
  data_unknown=subset(data,data[[o_col]] %in% unknown_class)

  data_known_b=list();
  data_known_train=list();
  data_known_valid=list();
  data_known_valid_overall=data.frame();
    
  ### CLASSWISE data split for training seperate A.D for each class
  
    for (j in 1:length(known_classes)){
    data_known_b[[j]]=subset(data,data[[o_col]] %in% known_classes[[j]])
    # data_unknown_b[[j]]=subset(data,data[[o_col]] %in% setdiff(tr,known_classes[j]))
    ####################  Now known_1 classes data split
    n_known_inst=nrow(data_known_b[[j]])
    ind_known_clas=1:nrow(data_known_b[[j]])
    data_known_b[[j]]$ind=ind_known_clas
    ind_known_t=sample(ind_known_clas,0.5*n_known_inst)
    data_known_train[[j]]=data_known_b[[j]][ind_known_t,1:n_col]  
    data_known_b[[j]]=data_known_b[[j]][-ind_known_t,1:n_col]

    ind_known_clas=1:nrow(data_known_b[[j]])
    data_known_b[[j]]$ind=ind_known_clas
    ind_known_t=sample(ind_known_clas,0.5*n_known_inst)
    data_known_valid[[j]]=data_known_b[[j]][ind_known_t,1:n_col]  
    data_known_b[[j]]=data_known_b[[j]][-ind_known_t,1:n_col]
    
    # names(data_known_valid_overall)=names(data_known_valid[[j]])
    data_known_valid_overall=rbind(data_known_valid_overall,data_known_valid[[j]])
   
  }
  ##########################

  n_known1=nrow(data_known_valid_overall)
  data_known=data_known_valid_overall
  n_unknown1=nrow(data_unknown)
  n_known_tobe=as.integer(((1-alpha)/alpha)*n_unknown1)

  if (n_known_tobe<n_known1){
    data_known=data_known[sample(1:n_known1,n_known_tobe,replace = FALSE),]
  }
  N_row1=nrow(data_known)
  N_anom=as.integer((alpha/(1-alpha))*(N_row1))
 ###############################
  ind_unknown_clas=1:nrow(data_unknown)
  data_unknown$ind=ind_unknown_clas
  ind_unknown=sample(ind_unknown_clas,min(N_anom,nrow(data_unknown)))  
  data_unknown_a=data_unknown[ind_unknown,1:n_col]

  if (N_anom>nrow(data_unknown)){
    print("not possible alpha in current setting")
  }

  n_unknown_inst=nrow(data_unknown_a)
  ind_unknown_clas=1:nrow(data_unknown_a)
  data_unknown_a$ind=ind_unknown_clas
  ind_test=sample(ind_unknown_clas,0.5*n_unknown_inst)  
  
  data_unknown_valid=data_unknown_a[ind_test,1:n_col]
  data_unknown_test=data_unknown_a[-ind_test,1:n_col]
  data_mix=rbind(data_unknown_valid,data_unknown_test,data_known_valid_overall)
  #######################################
  datab=c();loda_datab=c();
    
  n_test=nrow(data_mix)
  n_classes=length(known_classes)
  datan_t=data.frame(matrix(ncol = n_classes, nrow = n_test))
  
  
  loda_datan_t=data.frame(matrix(ncol = n_classes, nrow = n_test))
  loda_datan=c()
    
for  (j in 1:length(known_classes)){
  n=nrow(data_known_train[[j]])#for iforest file naming.
  k=j
  
  write.csv(data_known_train[[j]], file = paste("data1_",vpro,"_",n,"_", k, ".csv", sep = ""), row.names = FALSE)


system(paste('./iforest','-i', paste('./data1_',vpro,'_',n,'_',k,'.csv', sep = ""),'-o', paste('./depth1_',vpro,'_',n,'_',k,'.csv', sep = ""),'-s', round(0.2*n), '-t 1000 -g -k -b',paste('./forest1_',vpro,'_', k,'.bin',sep = "")), wait = TRUE)
score1 <- read.csv(paste('./depth1_',vpro,'_',n,'_',k,'.csv', sep = ""), header = TRUE)
datab_t <- as.numeric(unlist(score1))
system(paste('rm','-f',paste('./depth1_',vpro,'_',n,'_',k,'.csv', sep = "")))
  datab=c(datab,datab_t)
#     ## LODA CODE
#     bt_out = btloda(data_known_train[[j]],sparsity=NA, maxk=50, keep=NULL, exclude=NULL, debug=F,inf_replace = log(1e-09))
#     loda_datab_t= bt_out$oob_nll#loda oob from clean
#     loda_datab=c(loda_datab,loda_datab_t)
    
#     loda_datan_t[[j]]= get_neg_ll_all_hist(data_mix, bt_out$pvh$w, bt_out$pvh$hists, inf_replace = log(1e-09))

    system(paste('rm','-f',file = paste("data1_",vpro,"_",n,"_", k, ".csv", sep = ""), row.names = FALSE))
    }
    
#     loda_datan=apply(loda_datan_t,1,FUN=min)


for  (j in 1:length(known_classes)){
  n=nrow(data_known_train[[j]])#for iforest file naming.
  k=j
  
  write.csv(data_mix, file = paste("data2_",vpro,"_",n,"_", k, ".csv", sep = ""), row.names = FALSE)

# system(paste('./iforest','-i', paste('./data2_',vpro,'_',n,'_',k,'.csv', sep = ""),'-o', paste('./depth1_',vpro,'_',n,'_',k,'.csv', sep = ""),'-s', round(0.2*n), '-t 1000 -g -k -b',paste('./forest1_',vpro,'_', k,'.bin',sep = "")), wait = TRUE)
system(paste('./iforest','-i', paste('./data2_',vpro,'_',n,'_',k,'.csv', sep = "") ,'-o',paste('./depth2_',vpro,'_',n,'_',k,'.csv', sep = ""),'-g -f', paste('./forest1_',vpro,'_', k,'.bin',sep = "")), wait = TRUE)
score1 <- read.csv(paste('./depth2_',vpro,'_',n,'_',k,'.csv', sep = ""), header = TRUE)
datan_t1 <- as.numeric(unlist(score1))
system(paste('rm','-f',paste('./depth2_',vpro,'_',n,'_',k,'.csv', sep = "")))
datan_t[[j]]=datan_t1

system(paste('rm','-f',file = paste("data2_",vpro,"_",n,"_", k, ".csv", sep = ""), row.names = FALSE))
}
datan=apply(datan_t,1,FUN=min)

#############


    temp_perf1_e02=cv_result(datab, datan, alpha, 0.02)
    temp_perf1_e05=cv_result(datab, datan, alpha, 0.05)
    temp_perf1_e10=cv_result(datab, datan, alpha, 0.1)

#     loda_temp_perf1_e02=cv_result(loda_datab, loda_datan, alpha, 0.02)
#     loda_temp_perf1_e05=cv_result(loda_datab, loda_datan, alpha, 0.05)
#     loda_temp_perf1_e10=cv_result(loda_datab, loda_datan, alpha, 0.1)

    temp_perf2_e02=as.data.frame(t(c(colMeans(temp_perf1_e02[1:10,]),colMeans(temp_perf1_e02[11:20,]),colMeans(temp_perf1_e02[21:30,]),colMeans(temp_perf1_e02[31:40,]),colMeans(temp_perf1_e02[41:50,]),colMeans(temp_perf1_e02[51:60,]))))
    temp_perf2_e05=as.data.frame(t(c(colMeans(temp_perf1_e05[1:10,]),colMeans(temp_perf1_e05[11:20,]),colMeans(temp_perf1_e05[21:30,]),colMeans(temp_perf1_e05[31:40,]),colMeans(temp_perf1_e05[41:50,]),colMeans(temp_perf1_e05[51:60,]))))
    temp_perf2_e10=as.data.frame(t(c(colMeans(temp_perf1_e10[1:10,]),colMeans(temp_perf1_e10[11:20,]),colMeans(temp_perf1_e10[21:30,]),colMeans(temp_perf1_e10[31:40,]),colMeans(temp_perf1_e10[41:50,]),colMeans(temp_perf1_e10[51:60,]))))

#     loda_temp_perf2_e02=as.data.frame(t(c(colMeans(loda_temp_perf1_e02[1:10,]),colMeans(loda_temp_perf1_e02[11:20,]),colMeans(loda_temp_perf1_e02[21:30,]),colMeans(loda_temp_perf1_e02[31:40,]),colMeans(loda_temp_perf1_e02[41:50,]),colMeans(loda_temp_perf1_e02[51:60,]))))
#     loda_temp_perf2_e05=as.data.frame(t(c(colMeans(loda_temp_perf1_e05[1:10,]),colMeans(loda_temp_perf1_e05[11:20,]),colMeans(loda_temp_perf1_e05[21:30,]),colMeans(loda_temp_perf1_e05[31:40,]),colMeans(loda_temp_perf1_e05[41:50,]),colMeans(loda_temp_perf1_e05[51:60,]))))
#     loda_temp_perf2_e10=as.data.frame(t(c(colMeans(loda_temp_perf1_e10[1:10,]),colMeans(loda_temp_perf1_e10[11:20,]),colMeans(loda_temp_perf1_e10[21:30,]),colMeans(loda_temp_perf1_e10[31:40,]),colMeans(loda_temp_perf1_e10[41:50,]),colMeans(loda_temp_perf1_e10[51:60,]))))

    names(temp_perf2_e02)=names(comp_perf1_e02)
    names(temp_perf2_e05)=names(comp_perf1_e05)
    names(temp_perf2_e10)=names(comp_perf1_e10)

    comp_perf1_e02=rbind(comp_perf1_e02,temp_perf2_e02)
    comp_perf1_e05=rbind(comp_perf1_e05,temp_perf2_e05)
    comp_perf1_e10=rbind(comp_perf1_e10,temp_perf2_e10)

#     names(loda_temp_perf2_e02)=names(comp_perf1_loda_e02)
#     names(loda_temp_perf2_e05)=names(comp_perf1_loda_e05)
#     names(loda_temp_perf2_e10)=names(comp_perf1_loda_e10)

#     comp_perf1_loda_e02=rbind(comp_perf1_loda_e02,loda_temp_perf2_e02)
#     comp_perf1_loda_e05=rbind(comp_perf1_loda_e05,loda_temp_perf2_e05)
#     comp_perf1_loda_e10=rbind(comp_perf1_loda_e10,loda_temp_perf2_e10)
    
}#end runs


[1] "run"
[1] 1
[1] "OCR"


Warning message in file(file, "rt"):
“cannot open file './depth1_20_285_1.csv': No such file or directory”

ERROR: Error in file(file, "rt"): cannot open the connection


### We get the result's summary for various values of quantiles ( epsilon / q[paper] ) 

In [ ]:


overall_perf_e02=overall_perf(comp_perf1_e02,0.02)
overall_perf_e05=overall_perf(comp_perf1_e05,0.05)
overall_perf_e10=overall_perf(comp_perf1_e10,0.10)

# overall_perf_loda_e02=overall_perf(comp_perf1_loda_e02,0.02)
# overall_perf_loda_e05=overall_perf(comp_perf1_loda_e05,0.05)
# overall_perf_loda_e10=overall_perf(comp_perf1_loda_e10,0.10)


overall_performance=data.frame()

overall_performance=overall_perf_e02

names(overall_perf_e05)=names(overall_performance)
names(overall_perf_e10)=names(overall_performance)

# names(overall_perf_loda_e02)=names(overall_performance)
# names(overall_perf_loda_e05)=names(overall_performance)
# names(overall_perf_loda_e10)=names(overall_performance)

overall_performance=rbind(overall_performance,overall_perf_e05)
overall_performance=rbind(overall_performance,overall_perf_e10)

# overall_performance=rbind(overall_performance,overall_perf_loda_e02)
# overall_performance=rbind(overall_performance,overall_perf_loda_e05)
# overall_performance=rbind(overall_performance,overall_perf_loda_e10)

###### the final summary results are in this dataframe where the first 6 rows correspond to target quantile=0.02, then next 6 rows correspond to target quantile=0.05 and next 6 rows correspond to target quantile=0.1

In [ ]:
n1=nrow(comp_perf1_e02)
overall_performance_ci1=overall_performance_ci(overall_performance,n1)